In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

def fetchUrl(pid, uid, max_id):

    url = "https://weibo.com/ajax/statuses/buildComments"

    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    }

    params = {
        "flow" : 0,
        "is_reload" : 1,
        "id" : pid,
        "is_show_bulletin" : 2,
        "is_mix" : 0,
        "max_id" : max_id,
        "count" : 20,
        "uid" : uid,
    }

    r = requests.get(url, headers = headers, params = params)
    return r.json()

def parseJson(jsonObj):

    data = jsonObj["data"]
    max_id = jsonObj["max_id"]

    commentData = []
    for item in data:
        # 评论id
        comment_Id = item["id"]
        # 评论内容
        text = BeautifulSoup(item["text"], "html.parser").text
        # 评论时间
        created_at = item["created_at"]
        # 点赞数
        like_counts = item["like_counts"]
        # 评论数
        total_number = item["total_number"]

        # 评论者 id，name，gender, city, followers, friends
        user = item["user"]
        userID = user["id"]
        userName = user["screen_name"]
        userGender = user["gender"]
        userCity = user["location"]
        followers = user["followers_count"]
        following = user["friends_count"]

        dataItem = [comment_Id, created_at, userID, userName, userGender, userCity, like_counts, total_number, followers, following, text]
        print(dataItem)
        commentData.append(dataItem)

    return commentData, max_id

def save_data(data, path, filename):

    if not os.path.exists(path):
        os.makedirs(path)

    dataframe = pd.DataFrame(data)
    dataframe.to_csv(path + filename, encoding='utf_8_sig', mode='a', index=False, sep=',', header=False )

if __name__ == "__main__":

    pid = 4740355381529651       # 微博id，固定
    uid = 1784473157            # 用户id，固定
    max_id = 0
    path = "/Users/sallywong/Desktop"           # 保存的路径
    filename = "jsoncomments.csv"   # 保存的文件名
    
    csvHeader = [["cid", "publish_time", "uid", "username", "gender", "location", "like_count", "reply_count", "followers", "following", "text"]]
    save_data(csvHeader, path, filename)

    while(True):
        html = fetchUrl(pid, uid, max_id)
        comments, max_id = parseJson(html)
        save_data(comments, path, filename)
        # max_id 为 0 时，表示爬取结束
        if max_id == 0:
            break;


[4740355754561464, 'Thu Feb 24 11:12:26 +0800 2022', 6373141466, 'HYH吴情', 'f', '湖北 武汉', 38084, 1484, 629, 369, '俄乌局势，俄罗斯收了两个小碎片，美国赚的盆满钵满，乌克兰落得稀巴烂，欧盟个二缺也没讨到好，我们见证了历史，大家都有美好的未来']
[4740365099206476, 'Thu Feb 24 11:49:34 +0800 2022', 5939211207, '龍公子_Cheney', 'm', '浙江 杭州', 19428, 465, 1076035, 693, '巴赫说：感谢冬奥所争取的世界和平，原来是这个意思']
[4740355582853328, 'Thu Feb 24 11:11:44 +0800 2022', 2832392523, 'wydgo1993', 'm', '上海 普陀区', 6394, 687, 57, 355, '大的要来了吗？']
[4740356107143495, 'Thu Feb 24 11:13:50 +0800 2022', 3911031219, '丹心映山河', 'm', '湖北', 4698, 199, 3938, 8582, '近年来欧盟的主要国家法国和德国在力推组建欧盟军队，试图实现欧洲防务自主，减少对美国的依赖。这让美国很不爽，因为欧盟的目的如果实现，美国对欧洲的控制力也将随之显著下降。所以美国就不断炒作俄罗斯对乌克兰、对欧洲的威胁，不断增加在欧洲的驻军，以此加强对欧洲的控制。']
[4740355968206781, 'Thu Feb 24 11:13:17 +0800 2022', 6588070363, '问水行舟AQA', 'm', '其他', 4098, 127, 22, 194, '“不要再打了，求求你们不要了啦”']
[4740358635785930, 'Thu Feb 24 11:23:53 +0800 2022', 6522481188, '言午士心弓口虫', 'm', '天津', 3051, 243, 280822, 190, '你们打架，我加不起油了']
[4740356370859511, 'Thu Feb 24 11:14:53 +0800 2022', 62522

JSONDecodeError: Expecting value: line 1 column 1 (char 0)